# SELECT {fields} FROM {table} WHERE {condition}
- plus use of LIKE and wildcards in text fields

## Using SQL Server and Quering directly to de DB

In [62]:
### Connect to the BikeStores DB
import pyodbc

# Valid values for the connection string
driver = '{ODBC Driver 17 for SQL Server}'
server = '(local)'
dbname = 'AdventureWorks2019'
#dbname = 'BikeStores'
user = 'user1'
passwd = 'pass1'

# Construct the Connection String
connection_string = f'DRIVER={driver};SERVER={server};' +\
                    f'DATABASE={dbname};UID={user};PWD={passwd}'
print('Connection String:\n', connection_string)

# Establish the connection
try:
    connection = pyodbc.connect(connection_string)
    cursor = connection.cursor()
    print('SUCCESS: Connection Established')
except pyodbc.Error as e:
    print('ERROR:', e)

Connection String:
 DRIVER={ODBC Driver 17 for SQL Server};SERVER=(local);DATABASE=AdventureWorks2019;UID=user1;PWD=pass1
SUCCESS: Connection Established


In [63]:
### First look to the products table
query = ''' SELECT *
            FROM Production.Product '''
cursor.execute(query)
# -- at this point ready to close conn (cursor.close(); connection.close())

# Get data direct from cursor query execution
field_names = [i[0] for i in cursor.description]
get_data = [list(xx) for xx in cursor]
print(get_data)

[[1, 'Adjustable Race', 'AR-5381', False, False, None, 1000, 750, Decimal('0.0000'), Decimal('0.0000'), None, None, None, None, 0, None, None, None, None, None, datetime.datetime(2008, 4, 30, 0, 0), None, None, '694215B7-08F7-4C0D-ACB1-D734BA44C0C8', datetime.datetime(2014, 2, 8, 10, 1, 36, 827000)], [2, 'Bearing Ball', 'BA-8327', False, False, None, 1000, 750, Decimal('0.0000'), Decimal('0.0000'), None, None, None, None, 0, None, None, None, None, None, datetime.datetime(2008, 4, 30, 0, 0), None, None, '58AE3C20-4F3A-4749-A7D4-D568806CC537', datetime.datetime(2014, 2, 8, 10, 1, 36, 827000)], [3, 'BB Ball Bearing', 'BE-2349', True, False, None, 800, 600, Decimal('0.0000'), Decimal('0.0000'), None, None, None, None, 1, None, None, None, None, None, datetime.datetime(2008, 4, 30, 0, 0), None, None, '9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E', datetime.datetime(2014, 2, 8, 10, 1, 36, 827000)], [4, 'Headset Ball Bearings', 'BE-2908', False, False, None, 800, 600, Decimal('0.0000'), Decimal('0.0

In [64]:
### Construct the DF in base a list extracted
import pandas as pd

products_df = pd.DataFrame(data=get_data, columns=field_names)
display(products_df.iloc[[0, 5, -5, -1]])
products_df.columns

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,...,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.0000,...,None,None,None,NaN,NaN,2008-04-30,NaT,None,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
5,317,LL Crankarm,CA-5965,False,False,Black,500,375,0.0000,0.0000,...,None,L,None,NaN,NaN,2008-04-30,NaT,None,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.2400,...,None,M,None,5.0,96.0,2013-05-30,NaT,None,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827
503,999,"Road-750 Black, 52",BK-R19B-52,True,True,Black,100,75,343.6496,539.9900,...,R,L,U,2.0,31.0,2013-05-30,NaT,None,AE638923-2B67-4679-B90E-ABBAB17DCA31,2014-02-08 10:01:36.827


Index(['ProductID', 'Name', 'ProductNumber', 'MakeFlag', 'FinishedGoodsFlag',
       'Color', 'SafetyStockLevel', 'ReorderPoint', 'StandardCost',
       'ListPrice', 'Size', 'SizeUnitMeasureCode', 'WeightUnitMeasureCode',
       'Weight', 'DaysToManufacture', 'ProductLine', 'Class', 'Style',
       'ProductSubcategoryID', 'ProductModelID', 'SellStartDate',
       'SellEndDate', 'DiscontinuedDate', 'rowguid', 'ModifiedDate'],
      dtype='object')

In [76]:
### At this point I can query direct from the df but...
### I will continue with DBs query (only one more)

# Input an string to find in product name
prod_to_find = input('Enter a product to find:')
str_to_find = '%' + prod_to_find + '%'
#print(str_to_find)
query = ''' SELECT ProductID, Name, ListPrice, SellEndDate, DiscontinuedDate
            FROM Production.Product
            WHERE ListPrice > -1 AND Name LIKE ''' + "'" + str_to_find + "'"
print(query)
cursor.execute(query)
field_names = [i[0] for i in cursor.description]
get_data = [list(xx) for xx in cursor]
df = pd.DataFrame(data=get_data, columns=field_names)
df

 SELECT ProductID, Name, ListPrice, SellEndDate, DiscontinuedDate
            FROM Production.Product
            WHERE ListPrice > -1 AND Name LIKE '%bike%'


,ProductID,Name,ListPrice,SellEndDate,DiscontinuedDate
0,879,All-Purpose Bike Stand,159.0000,NaT,None
1,877,Bike Wash - Dissolver,7.9500,NaT,None
2,876,Hitch Rack - 4-Bike,120.0000,NaT,None
3,710,"Mountain Bike Socks, L",9.5000,2012-05-29,None
4,709,"Mountain Bike Socks, M",9.5000,2012-05-29,None
